## Querying the batch data

The code in this notebook was originally used following the code in the notebook [cleaning_batch_data.ipynb](cleaning_batch_data.ipynb)

### Query 1 - Find the most popular category in each country

In [ ]:
# Joins the geo and pin dataframes
pin_geo_df = pin_df.join(geo_df, pin_df["ind"] == geo_df["ind"], how="inner")

# Creates partition by country and order by category_count descending
window = Window.partitionBy("country").orderBy(col("category_count").desc())

# Finds the most popular category in each country
(pin_geo_df.groupBy("country", "category")
.agg(count("category").alias("category_count"))
.withColumn("rank", row_number().over(window))
.filter(col("rank") == 1)
.drop("rank")
.show())


+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Afghanistan| finance| 7|
 Albania|diy-and-crafts| 9|
 Algeria| quotes| 12|
 American Samoa| tattoos| 3|
 Andorra| tattoos| 3|
 Angola| finance| 1|
 Anguilla| education| 2|
Antarctica (the t...| christmas| 2|
 Antigua and Barbuda| travel| 6|
 Argentina| tattoos| 5|
 Armenia| christmas| 3|
 Aruba| tattoos| 6|
 Australia| mens-fashion| 2|
 Austria| travel| 1|
 Azerbaijan| finance| 4|
 Bahamas| art| 1|
 Bahrain| quotes| 4|
 Bangladesh| christmas| 3|
 Barbados| christmas| 1|
 Belarus| christmas| 1|
+--------------------+--------------+--------------+
only showing top 20 rows

### Query 2 - Find which was the most popular category each year between 2018 and 2022

In [ ]:
# Joins the geo and pin dataframes
pin_geo_df = pin_df.join(geo_df, pin_df["ind"] == geo_df["ind"], how="inner")

# Creates partition by post_year and order by category_count descending
window = Window.partitionBy("post_year").orderBy(col("category_count").desc())

# Finds which was the most popular category each year between 2018 and 2022
(pin_geo_df.withColumn("post_year", year("timestamp"))
.filter((col("post_year") >= 2018) & (col("post_year") <= 2022))
.groupBy("post_year", "category")
.agg(count("category").alias("category_count"))
.withColumn("rank", row_number().over(window))
.filter(col("rank") == 1)
.drop("rank")
.orderBy("post_year", "category_count")
.show())


+---------+--------------+--------------+
post_year| category|category_count|
+---------+--------------+--------------+
 2018| tattoos| 18|
 2019| tattoos| 19|
 2020| christmas| 20|
 2021|diy-and-crafts| 24|
 2022| finance| 13|
+---------+--------------+--------------+

### Query 3 - Find the user with the most followers in each country

#### Part 1 - For each country find the user with the most followers

In [ ]:
# Joins the geo and pin dataframes
pin_geo_df = pin_df.join(geo_df, pin_df["ind"] == geo_df["ind"], how="inner")

# Creates partition by country and order by follower_count descending
window = Window.partitionBy("country").orderBy(col("follower_count").desc())

# Finds the user with the most followers in each country
max_followers_by_country = (
    pin_geo_df.withColumn("rank", row_number().over(window))
    .filter(col("rank") == 1)
    .select("country", "poster_name", "follower_count"))

max_followers_by_country.show()


+--------------------+--------------------+--------------+
 country| poster_name|follower_count|
+--------------------+--------------------+--------------+
 Afghanistan| 9GAG| 3000000|
 Albania| The Minds Journal| 5000000|
 Algeria| Apartment Therapy| 5000000|
 American Samoa| Mamas Uncut| 8000000|
 Andorra| Our Mindful Life| 190000|
 Angola| Tastemade| 8000000|
 Anguilla| Hometalk.com| 5000000|
Antarctica (the t...| HikenDip| 500000|
 Antigua and Barbuda| A Cultivated Nest| 578000|
 Argentina| Who What Wear| 2000000|
 Armenia|Michelle {CraftyM...| 891000|
 Aruba| One Kindesign| 657000|
 Australia| Cultura Colectiva| 1000000|
 Austria|The World Pursuit...| 89000|
 Azerbaijan| SOCIETY19| 1000000|
 Bahamas| Stylecraze| 2000000|
 Bahrain| Decor Home Ideas| 144000|
 Bangladesh| Smart School House| 673000|
 Barbados|The Creativity Ex...| 410000|
 Belarus|Prudent Penny Pin...| 647000|
+--------------------+--------------------+--------------+
only showing top 20 rows

#### Part 2 - Based on the above query, find the country with the user with most followers.

In [ ]:
# Gest the highest number of followers of all countries
ultimate_max_followers = max_followers_by_country.select(max("follower_count")).collect()[0][0]

# Finds the country with the user with most followers
(max_followers_by_country.select("*")
 .where(col("follower_count") == ultimate_max_followers)
 .show())


+--------------+-----------+--------------+
 country|poster_name|follower_count|
+--------------+-----------+--------------+
American Samoa|Mamas Uncut| 8000000|
 Angola| Tastemade| 8000000|
+--------------+-----------+--------------+

### Query 4 - Find the most popular category in each age group

In [ ]:
# Joins the user and pin dataframes
pin_user_df = pin_df.join(user_df, pin_df["ind"] == user_df["ind"], how="inner")

# Creates the age_group column
pin_user_df_with_age = pin_user_df.withColumn("age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .otherwise("50+"))


# Creates partition by age_group and order by category_count descending
window = Window.partitionBy("age_group").orderBy(col("category_count").desc())

# Finds the most popular category in each age group
(pin_user_df_with_age.groupBy("age_group", "category")
.agg(count("category").alias("category_count"))
.withColumn("rank", row_number().over(window))
.filter(col("rank") == 1)
.drop("rank")
.show())


+---------+---------+--------------+
age_group| category|category_count|
+---------+---------+--------------+
 18-24| tattoos| 35|
 25-35|christmas| 24|
 36-50|christmas| 12|
 50+| finance| 8|
+---------+---------+--------------+

### Query 5 - Find the median follower count of users in each age group

In [ ]:
# Joins the user and pin dataframes
pin_user_df = pin_df.join(user_df, pin_df["ind"] == user_df["ind"], how="inner")


# Creates the age_group column
pin_user_df_with_age = pin_user_df.withColumn("age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .otherwise("50+"))


# Finds the median follower count of users in each age group
(pin_user_df_with_age.groupBy("age_group")
.agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))
.orderBy("age_group")
.show())


+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 18-24| 226000|
 25-35| 18000|
 36-50| 19000|
 50+| 5000|
+---------+---------------------+

### Query 6 - Find how many users have joined between 2015 and 2020

In [ ]:
# Finds out how many users joined each year between 2015 and 2020
(user_df.withColumn("join_year", year("date_joined"))
.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))
.groupBy("join_year")
.agg(count("user_name").alias("number_of_users_joined"))
.orderBy("join_year")
.show())


+---------+----------------------+
join_year|number_of_users_joined|
+---------+----------------------+
 2015| 213|
 2016| 230|
 2017| 81|
+---------+----------------------+

### Query 7 - Find the median follower count of users that joined between 2015 and 2020

In [ ]:
# Joins the user and pin dataframes
pin_user_df = pin_df.join(user_df, pin_df["ind"] == user_df["ind"], how="inner")

# Finds the median follower count of users that joined between 2015 and 2020
(pin_user_df.withColumn("join_year", year("date_joined"))
.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))
.groupBy("join_year")
.agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))
.orderBy("join_year")
.show())


+---------+---------------------+
join_year|median_follower_count|
+---------+---------------------+
 2015| 211000|
 2016| 17000|
 2017| 5000|
+---------+---------------------+

### Query 8 - Find the median follower count of users that joined between 2015 and 2020 in each age group

In [ ]:
# Joins the user and pin dataframes
pin_user_df = pin_df.join(user_df, pin_df["ind"] == user_df["ind"], how="inner")

# Creates the age_group column
pin_user_df_with_age = pin_user_df.withColumn("age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .otherwise("50+"))

# Finds the median follower count of users that joined between 2015 and 2020 in each age group
(pin_user_df_with_age.withColumn("join_year", year("date_joined"))
.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))
.groupBy("age_group", "join_year")
.agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))
.orderBy("age_group", "join_year")
.show())


+---------+---------+---------------------+
age_group|join_year|median_follower_count|
+---------+---------+---------------------+
 18-24| 2015| 472000|
 18-24| 2016| 19000|
 18-24| 2017| 476|
 25-35| 2015| 33000|
 25-35| 2016| 16000|
 25-35| 2017| 3000|
 36-50| 2015| 24000|
 36-50| 2016| 17000|
 36-50| 2017| 8000|
 50+| 2015| 14000|
 50+| 2016| 3000|
 50+| 2017| 10000|
+---------+---------+---------------------+